In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 1.6 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# @title import library

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# @title import dataset

data = pd.read_csv(
    "https://raw.githubusercontent.com/miguroi/bcc-intern/main/Agrofood_co2_emission.csv"
    )

data.rename(columns={'Area':'area', 'Year':'year', 'Savanna fires':'savanna_fires', 'Forest fires':'forest_fires', 'Crop Residues':'crop_residues', 'Rice Cultivation':'rice_cultivation', 'Drained organic soils (CO2)': 'co2', 'Pesticides Manufacturing': 'pesticides_manufacture', 'Food Transport': 'food_transport', 'Forestland':'forestland', 'Net Forest conversion': 'net_forest_conversion', 'Food Household Consumption': 'food_household_consumption', 'Food Retail':'food_retail', 'On-farm Electricity Use':'on_farm_electricity', 'Food Packaging':'food_packaging', 'Agrifood Systems Waste Disposal': 'agrifood_systems_waste_disposal', 'Food Processing':'food_processing', 'Fertilizers Manufacturing':'fertilizers_manufacture', 'IPPU':'ippu', 'Manure applied to Soils':'manure_soils', 'Manure left on Pasture':'manure_pasture', 'Manure Management':'manure_management', 'Fires in organic soils':'fires_organic_soils', 'Fires in humid tropical forests':'fires_tropical_forests', 'On-farm energy use':'on_farm_energy', 'Rural population':'rural_population', 'Urban population':'urban_population', 'Total Population - Male':'total_male', 'Total Population - Female':'total_female', 'Average Temperature °C':'average_temperature'}, inplace=True)

data.describe()

In [ ]:
# @title grouping by sectors

total_sectors = {
    'Kebakaran': data[['savanna_fires', 'forest_fires', 'fires_organic_soils', 'fires_tropical_forests']].sum(axis=1),
    'Pertanian': data[['crop_residues', 'rice_cultivation', 'co2', 'manure_soils', 'manure_pasture', 'manure_management']].sum(axis=1),
    'Industri Pertanian': data[['fertilizers_manufacture', 'pesticides_manufacture']].sum(axis=1),
    'Transportasi dan Distribusi Makanan': data[['food_transport', 'food_retail', 'food_packaging']].sum(axis=1),
    'Konsumsi Makanan': data['food_household_consumption'],
    'Pengolahan Makanan': data['food_processing'],
    'Industri': data['ippu'],
    'Pengolahan Limbah Pertanian': data['agrifood_systems_waste_disposal']
}

total_sectors_df = pd.DataFrame(total_sectors)

In [ ]:
data = pd.concat([data, total_sectors_df], axis=1)

data.describe()

In [ ]:
# @title grouping by continents

def group_countries_by_continent(countries):
    continents = {
        'Africa': [],
        'Asia': [],
        'Europe': [],
        'North America': [],
        'Oceania': [],
        'South America': [],
        'Other': []
    }

    country_to_continent = {
        # Africa
        'Morocco':'Africa', 'Mozambique':'Africa', 'Namibia':'Africa', 'Niger':'Africa', 'Nigeria':'Africa',
        'Rwanda':'Africa', 'Saint Helena, Ascension and Tristan da Cunha':'Africa', 'Sao Tome and Principe':'Africa',
        'Senegal':'Africa', 'Seychelles':'Africa', 'Sierra Leone':'Africa', 'Somalia':'Africa', 'South Africa':'Africa',
        'South Sudan':'Africa', 'Sudan':'Africa', 'Eswatini':'Africa', 'Tanzania':'Africa', 'Togo':'Africa',
        'Tunisia':'Africa', 'Uganda':'Africa', 'Western Sahara':'Africa', 'Zambia':'Africa', 'Zimbabwe':'Africa',
        'Madagascar':'Africa', 'Malawi':'Africa', 'Mali':'Africa', 'Mauritania':'Africa', 'Mauritius':'Africa',
        'Mayotte':'Africa', 'Lesotho':'Africa', 'Liberia':'Africa', 'Libya':'Africa',

        # Asia
        'Afghanistan':'Asia', 'Armenia':'Asia', 'Azerbaijan':'Asia', 'Bahrain':'Asia', 'Bangladesh':'Asia',
        'Bhutan':'Asia', 'Brunei Darussalam':'Asia', 'Cambodia':'Asia', 'China':'Asia', 'China, Hong Kong SAR':'Asia',
        'China, Macao SAR':'Asia', 'China, mainland':'Asia', 'China, Taiwan Province of':'Asia', 'Cyprus':'Asia',
        'Georgia':'Asia', 'India':'Asia', 'Indonesia':'Asia', 'Iran (Islamic Republic of)':'Asia', 'Iraq':'Asia',
        'Israel':'Asia', 'Japan':'Asia', 'Jordan':'Asia', 'Kazakhstan':'Asia', 'Kuwait':'Asia', 'Kyrgyzstan':'Asia',
        'Lao People\'s Democratic Republic':'Asia', 'Lebanon':'Asia', 'Malaysia':'Asia', 'Maldives':'Asia',
        'Mongolia':'Asia', 'Myanmar':'Asia', 'Nepal':'Asia', 'Oman':'Asia', 'Pakistan':'Asia', 'Palestine':'Asia',
        'Philippines':'Asia', 'Qatar':'Asia', 'Republic of Korea':'Asia', 'Russian Federation':'Asia',
        'Saudi Arabia':'Asia', 'Singapore':'Asia', 'Sri Lanka':'Asia', 'Syrian Arab Republic':'Asia', 'Tajikistan':'Asia',
        'Thailand':'Asia', 'Timor-Leste':'Asia', 'Turkey':'Asia', 'Turkmenistan':'Asia', 'United Arab Emirates':'Asia',
        'Uzbekistan':'Asia', 'Viet Nam':'Asia', 'Yemen':'Asia',

        # Europe
        'Albania':'Europe', 'Andorra':'Europe', 'Austria':'Europe', 'Belarus':'Europe', 'Belgium':'Europe',
        'Bosnia and Herzegovina':'Europe', 'Bulgaria':'Europe', 'Croatia':'Europe', 'Czechia':'Europe',
        'Denmark':'Europe', 'Estonia':'Europe', 'Finland':'Europe', 'France':'Europe', 'Germany':'Europe',
        'Greece':'Europe', 'Hungary':'Europe', 'Iceland':'Europe', 'Ireland':'Europe', 'Italy':'Europe',
        'Latvia':'Europe', 'Lithuania':'Europe', 'Luxembourg':'Europe', 'Malta':'Europe', 'Monaco':'Europe',
        'Montenegro':'Europe', 'Netherlands':'Europe', 'North Macedonia':'Europe', 'Norway':'Europe', 'Poland':'Europe',
        'Portugal':'Europe', 'Romania':'Europe', 'San Marino':'Europe', 'Serbia':'Europe', 'Slovakia':'Europe',
        'Slovenia':'Europe', 'Spain':'Europe', 'Sweden':'Europe', 'Switzerland':'Europe', 'Ukraine':'Europe',
        'United Kingdom of Great Britain and Northern Ireland':'Europe', 'Isle of Man':'Europe',

        # North America
        'Antigua and Barbuda':'North America', 'Bahamas':'North America', 'Barbados':'North America', 'Belize':'North America',
        'Canada':'North America', 'Costa Rica':'North America', 'Cuba':'North America', 'Dominica':'North America',
        'Dominican Republic':'North America', 'El Salvador':'North America', 'Grenada':'North America', 'Guatemala':'North America',
        'Haiti':'North America', 'Honduras':'North America', 'Jamaica':'North America', 'Mexico':'North America',
        'Nicaragua':'North America', 'Panama':'North America', 'Saint Kitts and Nevis':'North America', 'Saint Lucia':'North America',
        'Saint Vincent and the Grenadines':'North America', 'Trinidad and Tobago':'North America', 'United States of America':'North America',

        # Oceania
        'Australia':'Oceania', 'Fiji':'Oceania', 'Kiribati':'Oceania', 'Marshall Islands':'Oceania',
        'Micronesia (Federated States of)':'Oceania', 'Nauru':'Oceania', 'New Zealand':'Oceania', 'Palau':'Oceania',
        'Papua New Guinea':'Oceania', 'Samoa':'Oceania', 'Solomon Islands':'Oceania', 'Tonga':'Oceania',
        'Tuvalu':'Oceania', 'Vanuatu':'Oceania',

        # South America
        'Argentina':'South America', 'Bolivia (Plurinational State of)':'South America', 'Brazil':'South America',
        'Chile':'South America', 'Colombia':'South America', 'Ecuador':'South America', 'Guyana':'South America',
        'Paraguay':'South America', 'Peru':'South America', 'Suriname':'South America', 'Uruguay':'South America',
        'Venezuela (Bolivarian Republic of)':'South America',

        # Other
        'Montserrat':'Other', 'Niue':'Other', 'Puerto Rico':'Other', 'Tokelau':'Other',
        'United States Virgin Islands':'Other', 'Wallis and Futuna Islands':'Other', 'Falkland Islands (Malvinas)':'Other',
        'French Polynesia':'Other', 'Greenland':'Other', 'Guam':'Other', 'American Samoa':'Other', 'Aruba':'Other',
        'Bermuda':'Other', 'British Virgin Islands':'Other', 'Cayman Islands':'Other', 'Cook Islands':'Other',
        'Curaçao':'Other', 'Faroe Islands':'Other', 'French Guiana':'Other', 'Gibraltar':'Other', 'Guadeloupe':'Other',
        'Martinique':'Other', 'Mayotte':'Other', 'Montenegro':'Other', 'New Caledonia':'Other', 'Pitcairn':'Other',
        'Réunion':'Other', 'Saint Barthélemy':'Other', 'Saint Martin (French part)':'Other', 'Sint Maarten (Dutch part)':'Other',
        'South Georgia and the South Sandwich Islands':'Other', 'Svalbard and Jan Mayen Islands':'Other', 'Turks and Caicos Islands':'Other'
    }

    for country in countries:
        continent = country_to_continent.get(country, 'Other')
        continents[continent].append(country)

    return continents

countries = [
    'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda',
    'Saint Helena, Ascension and Tristan da Cunha', 'Sao Tome and Principe',
    'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa',
    'South Sudan', 'Sudan', 'Eswatini', 'Tanzania', 'Togo', 'Tunisia',
    'Uganda', 'Western Sahara', 'Zambia', 'Zimbabwe', 'Madagascar', 'Malawi',
    'Mali', 'Mauritania', 'Mauritius', 'Mayotte', 'Lesotho', 'Liberia', 'Libya',

    # Asia
    'Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei Darussalam', 'Cambodia',
    'China', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland', 'China, Taiwan Province of', 'Cyprus',
    'Georgia', 'India', 'Indonesia', 'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan',
    'Kuwait', 'Kyrgyzstan', 'Lao People\'s Democratic Republic', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia',
    'Myanmar', 'Nepal', 'Oman', 'Pakistan', 'Palestine', 'Philippines', 'Qatar', 'Republic of Korea', 'Russian Federation',
    'Saudi Arabia', 'Singapore', 'Sri Lanka', 'Syrian Arab Republic', 'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey',
    'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Viet Nam', 'Yemen',

    # Europe
    'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czechia',
    'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
    'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway',
    'Poland', 'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
    'Ukraine', 'United Kingdom of Great Britain and Northern Ireland', 'Isle of Man',

    # North America
    'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic',
    'El Salvador', 'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama',
    'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States of America',

    # Oceania
    'Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia (Federated States of)', 'Nauru', 'New Zealand', 'Palau',
    'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu',

    # South America
    'Argentina', 'Bolivia (Plurinational State of)', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay',
    'Peru', 'Suriname', 'Uruguay', 'Venezuela (Bolivarian Republic of)',

    # Other territories and regions (islands and territories)
    'Montserrat', 'Niue', 'Puerto Rico', 'Tokelau', 'United States Virgin Islands', 'Wallis and Futuna Islands',
    'Falkland Islands (Malvinas)', 'French Polynesia', 'Greenland', 'Guam', 'American Samoa', 'Aruba', 'Bermuda',
    'British Virgin Islands', 'Cayman Islands', 'Cook Islands', 'Curaçao', 'Faroe Islands', 'French Guiana', 'Gibraltar',
    'Guadeloupe', 'Martinique', 'Mayotte', 'Montenegro', 'New Caledonia', 'Pitcairn', 'Réunion', 'Saint Barthélemy',
    'Saint Martin (French part)', 'Sint Maarten (Dutch part)', 'South Georgia and the South Sandwich Islands', 'Svalbard and Jan Mayen Islands',
    'Turks and Caicos Islands'
]

grouped_countries = group_countries_by_continent(countries)

for continent, countries_list in grouped_countries.items():
    print(f'{continent}: {", ".join(countries_list)}')

In [ ]:
# @title grouping by continents (continue)

# bikin list countries dari area
countries = data['area'].unique()

# bikin grouping negara berdasarkan benua
grouped_countries = group_countries_by_continent(countries)

# bikin kolom 'continent' di data
data['continent'] = ''

# nambahin value buat kolom 'continent' berdasarkan dictionary 'grouped_countries'
for index, row in data.iterrows():
    country = row['area']
    continent = None
    for key, value in grouped_countries.items():
        if country in value:
            continent = key
            break
    data.at[index, 'continent'] = continent

In [ ]:
# @title cleaning using knnimputer

imputer = KNNImputer(n_neighbors=5, weights="uniform")

imputed_columns = data.select_dtypes(include=['float64', 'int64']).columns # impute kolom-kolom numerik

data[imputed_columns] = imputer.fit_transform(data[imputed_columns])

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

X_cluster = data

In [ ]:
# @title encoding object columns

kolom_objek = data.select_dtypes(include=object).columns.tolist()

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

data_ohe = ohe.fit_transform(data[kolom_objek]).toarray()
data_ohe = pd.DataFrame(data_ohe, columns=ohe.get_feature_names_out())

data = pd.concat([data, data_ohe], axis=1)
data = data.drop(kolom_objek, axis=1)

In [ ]:
X_cluster = X_cluster.select_dtypes(include=[np.number]).copy()

hapus = ['fires_organic_soils', 'fires_tropical_forests', 'savanna_fires',
                 'ippu','pesticides_manufacture', 'food_processing', 'fertilizers_manufacture','food_transport','food_packaging', 'food_retail',
                 'rice_cultivation', 'manure_pasture', 'crop_residues', 'manure_soils','co2','manure_management','agrifood_systems_waste_disposal',
                 'food_household_consumption']

X_cluster.drop(columns=hapus, inplace=True)

X_cluster.head()


In [ ]:
# @title mengecek metode clustering

# from sklearn.metrics import silhouette_score
# from sklearn.cluster import KMeans
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# from sklearn.metrics import davies_bouldin_score
# from sklearn.cluster import KMeans

# X = data_train.select_dtypes(include=[np.number]).copy()

k_range = range(2, 30)

silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k)
    cluster_labels = kmeans.fit_predict(X_cluster)
    silhouette_scores.append(silhouette_score(X_cluster, cluster_labels))

plt.plot(k_range, silhouette_scores)
plt.xlabel('Jumlah cluster')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.show()

In [ ]:
# @title Clustering without PCA

n_clusters = 7

# nerapin algoritma KMeans pada data tanpa PCA
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_cluster)
cluster_labels = kmeans.labels_

# hitung silhouette score
silhouette_score_without_pca = silhouette_score(X_cluster, cluster_labels)

# ngehitung jumlah cluster
cluster_counts = pd.Series(cluster_labels).value_counts()

print("Silhouette Score without PCA:", silhouette_score_without_pca)
print("Cluster Counts:\n", cluster_counts)


In [ ]:
# @title clustering all features using PCA & kmeans

n_clusters = 7

# apply pca buat mengurangi dimensionality sebelum clustering
pca = PCA(n_components=0.95)  # jaga 95% of variance
X_pca = pca.fit_transform(X_cluster)

# kmeans buat cluster data yang udah ditransformasi pakai PCA
kmeans_pca = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_pca.fit(X_pca)
pca_cluster_labels = kmeans_pca.labels_

# ngehitung silhouette score
pca_silhouette_score = silhouette_score(X_pca, pca_cluster_labels)

# ngehitung cluster counts
pca_cluster_counts = pd.Series(pca_cluster_labels).value_counts()

print("PCA Silhouette Score:", pca_silhouette_score)
print("PCA Cluster Counts:\n", pca_cluster_counts)

In [ ]:
import matplotlib.pyplot as plt

# Visualisasi clustering menggunakan PCA
plt.figure(figsize=(8, 6))

# Plot semua titik data dengan warna berdasarkan label kluster
plt.scatter(X_pca[:, 0], X_pca[:, 0], c=pca_cluster_labels, cmap='viridis', s=50, alpha=0.5)

# Plot pusat kluster
plt.scatter(kmeans_pca.cluster_centers_[:, 0], kmeans_pca.cluster_centers_[:, 0], c='red', marker='x', s=200, label='Centroids')

plt.title('Clustering using PCA and KMeans')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 1')
plt.legend()
plt.colorbar(label='Cluster')

plt.show()


In [ ]:
centroids = kmeans_pca.cluster_centers_
print("Cluster Centroids:\n", centroids)

# add cluster labels to X_cluster dataframe
X_cluster['Cluster'] = pca_cluster_labels

# compute mean/median values for each feature within each cluster
cluster_profiles_mean = X_cluster.groupby('Cluster').mean()
cluster_profiles_median = X_cluster.groupby('Cluster').median()

print("Cluster Profiles (Mean):\n", cluster_profiles_mean)
print("\nCluster Profiles (Median):\n", cluster_profiles_median)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import parallel_coordinates
import seaborn as sns

data_for_vis = cluster_profiles_mean.reset_index()

# Parallel Coordinates Plot
plt.figure(figsize=(50, 8))
parallel_coordinates(data_for_vis, 'Cluster', colormap=plt.get_cmap("Set2"))
plt.title('Parallel Coordinates Plot for Cluster Centroids')
plt.show()

# Heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(cluster_profiles_mean.drop('year', axis=1).T, cmap='YlGnBu', annot=True)
plt.title('Heatmap of Cluster Profiles')
plt.show()

# Box Plot
plt.figure(figsize=(14, 8))
sns.boxplot(x='Cluster', y='total_emission', data=X_cluster)
plt.title('Box Plot of Total Emission by Cluster')
plt.show()

# Box Plot
plt.figure(figsize=(14, 8))
sns.boxplot(x='Cluster', y='average_temperature', data=X_cluster)
plt.title('Box Plot of Total Emission by Cluster')
plt.show()

In [ ]:
# @title prepare data for modelling

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

y = data['average_temperature']
X = data.drop(['average_temperature'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# @title model testing

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
svr_model = SVR(kernel='rbf')
linear_model = LinearRegression()

# training models
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
svr_model.fit(X_train, y_train)
linear_model.fit(X_train, y_train)

# prediksikan test set
y_pred_rf = rf_model.predict(X_test)
y_pred_gb = gb_model.predict(X_test)
y_pred_svr = svr_model.predict(X_test)
y_pred_linear = linear_model.predict(X_test)

# evaluasi model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print("Random Forest:")
print(f"MSE: {mse_rf}, R^2: {r2_rf}")
print()

print("Gradient Boosting:")
print(f"MSE: {mse_gb}, R^2: {r2_gb}")
print()

print("Support Vector Regression:")
print(f"MSE: {mse_svr}, R^2: {r2_svr}")
print()

print("Linear Regression:")
print(f"MSE: {mse_linear}, R^2: {r2_linear}")
print()

In [ ]:
# @title checking for overfitting/target skewness

# ngehitung mse buat train set
train_mse_rf = mean_squared_error(y_train, rf_model.predict(X_train))
train_mse_gb = mean_squared_error(y_train, gb_model.predict(X_train))
train_mse_svr = mean_squared_error(y_train, svr_model.predict(X_train))
train_mse_linear = mean_squared_error(y_train, linear_model.predict(X_train))

# ngehitung perbedaan performa antara training dan testing
performance_difference_rf = train_mse_rf - mse_rf
performance_difference_gb = train_mse_gb - mse_gb
performance_difference_svr = train_mse_svr - mse_svr
performance_difference_linear = train_mse_linear - mse_linear

def check_overfitting(performance_difference):
    return abs(performance_difference) > 0.01

overfitting_rf = check_overfitting(performance_difference_rf)
overfitting_gb = check_overfitting(performance_difference_gb)
overfitting_svr = check_overfitting(performance_difference_svr)
overfitting_linear = check_overfitting(performance_difference_linear)

results = {
    "Random Forest": {"Train MSE": train_mse_rf, "Test MSE": mse_rf, "Difference": performance_difference_rf, "Overfitting": overfitting_rf},
    "Gradient Boosting": {"Train MSE": train_mse_gb, "Test MSE": mse_gb, "Difference": performance_difference_gb, "Overfitting": overfitting_gb},
    "SVR": {"Train MSE": train_mse_svr, "Test MSE": mse_svr, "Difference": performance_difference_svr, "Overfitting": overfitting_svr},
    "Linear Regression": {"Train MSE": train_mse_linear, "Test MSE": mse_linear, "Difference": performance_difference_linear, "Overfitting": overfitting_linear}
}

for model_name, metrics in results.items():
    print(model_name)
    print(metrics)
    print()

In [ ]:
from sklearn.model_selection import cross_val_score

# jumlah folds buat cross-validation
cv_folds = 5

# perform cross-validation for each model
cv_mse_rf = -cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds).mean()
cv_mse_gb = -cross_val_score(gb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds).mean()
cv_mse_svr = -cross_val_score(svr_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds).mean()
cv_mse_linear = -cross_val_score(linear_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds).mean()

cv_mse_rf, cv_mse_gb, cv_mse_svr, cv_mse_linear


In [ ]:
from sklearn.model_selection import cross_val_score

# Jumlah folds untuk cross-validation
cv_folds = 10

# Melakukan cross-validation untuk setiap model
cv_scores_rf = -cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds)
cv_scores_gb = -cross_val_score(gb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds)
cv_scores_svr = -cross_val_score(svr_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds)
cv_scores_linear = -cross_val_score(linear_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_folds)

# Mencetak hasil cross-validation
print("Cross-Validation Scores RF:")
for i, score in enumerate(cv_scores_rf, 1):
    print(f"Fold {i}: {score}")

print("\nCross-Validation Scores GB:")
for i, score in enumerate(cv_scores_gb, 1):
    print(f"Fold {i}: {score}")

print("\nCross-Validation Scores SVR:")
for i, score in enumerate(cv_scores_svr, 1):
    print(f"Fold {i}: {score}")

print("\nCross-Validation Scores Linear:")
for i, score in enumerate(cv_scores_linear, 1):
    print(f"Fold {i}: {score}")

    import numpy as np

# Menghitung rata-rata dan standar deviasi dari setiap skor cross-validation untuk setiap model
mean_cv_score_rf = np.mean(cv_scores_rf)
std_cv_score_rf = np.std(cv_scores_rf)

mean_cv_score_gb = np.mean(cv_scores_gb)
std_cv_score_gb = np.std(cv_scores_gb)

mean_cv_score_svr = np.mean(cv_scores_svr)
std_cv_score_svr = np.std(cv_scores_svr)

mean_cv_score_linear = np.mean(cv_scores_linear)
std_cv_score_linear = np.std(cv_scores_linear)

# Mencetak rata-rata dan standar deviasi dari setiap skor cross-validation untuk setiap model
print("Mean Cross-Validation Scores RF:", mean_cv_score_rf)
print("Standard Deviation Cross-Validation Scores RF:", std_cv_score_rf)

print("\nMean Cross-Validation Scores GB:", mean_cv_score_gb)
print("Standard Deviation Cross-Validation Scores GB:", std_cv_score_gb)

print("\nMean Cross-Validation Scores SVR:", mean_cv_score_svr)
print("Standard Deviation Cross-Validation Scores SVR:", std_cv_score_svr)

print("\nMean Cross-Validation Scores Linear:", mean_cv_score_linear)
print("Standard Deviation Cross-Validation Scores Linear:", std_cv_score_linear)


# unused data

In [ ]:
import seaborn as sns

data_vis = data[['urban_population', 'total_emission']]
data_vis['Cluster'] = pca_cluster_labels

sns.pairplot(data_vis, hue='Cluster', palette='viridis', diag_kind='kde')
plt.show()

from scipy.cluster.hierarchy import dendrogram, linkage

ahc = linkage(X_cluster, method='ward')

plt.figure(figsize=(15, 10))
dendrogram(ahc, truncate_mode='level', p=2)
plt.title('Dendrogram for Agglomerative Hierarchical Clustering')
plt.xlabel('Data points')
plt.ylabel('Euclidean distance')
plt.show()